In [1]:

!pip install torch torchvision


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 91.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [2]:
from google.colab import files

uploaded = files.upload()


Saving cnn_model.py to cnn_model.py


In [3]:
from google.colab import files

uploaded = files.upload()

Saving affectnet_dataset.py to affectnet_dataset.py


In [4]:
from google.colab import files

uploaded = files.upload()

Saving train.py to train.py


In [5]:
# ZIP yükle
uploaded = files.upload()

# ZIP aç
import zipfile
import os

with zipfile.ZipFile("YOLO_format.zip", 'r') as zip_ref:
    zip_ref.extractall("YOLO_format")


Saving YOLO_format.zip to YOLO_format.zip


In [9]:
# train.py (Colab için güçlü sürüm)

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
from cnn_model import EmotionCNN
from affectnet_dataset import AffectNetYOLODataset
from pathlib import Path
import time
import os

# 📁 Temel klasör
base_path = Path(".")

# 🔌 Cihaz seçimi
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("🖥️ Kullanılan cihaz:", device)

# 🔄 Dönüşümler
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

# 📂 Dizinler
train_image_dir = base_path / "YOLO_format/YOLO_format/train/images"
train_label_dir = base_path / "YOLO_format/YOLO_format/train/labels"
val_image_dir = base_path / "YOLO_format/YOLO_format/valid/images"
val_label_dir = base_path / "YOLO_format/YOLO_format/valid/labels"

# 🧾 Dataset ve Loader
train_dataset = AffectNetYOLODataset(train_image_dir, train_label_dir, transform)
val_dataset = AffectNetYOLODataset(val_image_dir, val_label_dir, transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=32, num_workers=2)

# 🧠 Model
model = EmotionCNN(num_classes=8).to(device)

# ⚙️ Kayıp + optimizer + scheduler
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0003)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.8)

# 📈 Eğitim ayarları
num_epochs = 50
best_val_acc = 0.0
log_path = base_path / "training_log.txt"
model_best_path = base_path / "emotion_cnn_best.pth"
model_last_path = base_path / "emotion_cnn_last.pth"

if os.path.exists(log_path):
    os.remove(log_path)

print(f"\n🚀 Eğitim başlıyor ({num_epochs} epoch):\n")
start_time = time.time()

# 🔁 Eğitim döngüsü
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    correct, total = 0, 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    train_acc = 100 * correct / total
    avg_loss = total_loss / len(train_loader)

    # 🧪 Doğrulama
    model.eval()
    val_correct, val_total = 0, 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            val_correct += (predicted == labels).sum().item()
            val_total += labels.size(0)

    val_acc = 100 * val_correct / val_total
    scheduler.step()

    log_line = f"📊 Epoch [{epoch+1}/{num_epochs}] | Loss: {avg_loss:.4f} | Train Acc: {train_acc:.2f}% | Val Acc: {val_acc:.2f}%"
    print(log_line)

    # Log'a yaz
    with open(log_path, "a") as f:
        f.write(log_line + "\n")

    # En iyi modeli kaydet
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), model_best_path)
        print(f"💾 Yeni en iyi model kaydedildi: {model_best_path.name} ({val_acc:.2f}%)")

# ⏱️ Eğitim süresi
elapsed = time.time() - start_time
print(f"\n✅ Eğitim tamamlandı! Toplam süre: {elapsed/60:.2f} dakika")

# 🔚 Son modeli kaydet
torch.save(model.state_dict(), model_last_path)
print(f"💾 Son model kaydedildi: {model_last_path.name}")


🖥️ Kullanılan cihaz: cuda

🚀 Eğitim başlıyor (50 epoch):

📊 Epoch [1/50] | Loss: 2.3264 | Train Acc: 15.34% | Val Acc: 23.31%
💾 Yeni en iyi model kaydedildi: emotion_cnn_best.pth (23.31%)
📊 Epoch [2/50] | Loss: 2.0119 | Train Acc: 19.57% | Val Acc: 26.95%
💾 Yeni en iyi model kaydedildi: emotion_cnn_best.pth (26.95%)
📊 Epoch [3/50] | Loss: 1.9833 | Train Acc: 21.43% | Val Acc: 26.75%
📊 Epoch [4/50] | Loss: 1.9834 | Train Acc: 21.19% | Val Acc: 27.62%
💾 Yeni en iyi model kaydedildi: emotion_cnn_best.pth (27.62%)
📊 Epoch [5/50] | Loss: 1.9803 | Train Acc: 21.41% | Val Acc: 27.16%
📊 Epoch [6/50] | Loss: 1.9790 | Train Acc: 21.23% | Val Acc: 27.49%
📊 Epoch [7/50] | Loss: 1.9710 | Train Acc: 21.47% | Val Acc: 26.30%
📊 Epoch [8/50] | Loss: 1.9745 | Train Acc: 21.24% | Val Acc: 27.41%
📊 Epoch [9/50] | Loss: 1.9734 | Train Acc: 21.35% | Val Acc: 27.89%
💾 Yeni en iyi model kaydedildi: emotion_cnn_best.pth (27.89%)
📊 Epoch [10/50] | Loss: 1.9633 | Train Acc: 21.68% | Val Acc: 27.47%
📊 Epoch [11/5

In [11]:
from google.colab import files

# En iyi modeli indir
files.download("emotion_cnn_best.pth")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>